# Matrikel-Nr: 2235021

# **Painting Classification**

Classify the painter with labeld images of paintings. Implementation with convolutional neural networks & transfer learning.

## **Imports**

In [5]:
import os
from PIL import Image
import numpy as np
import re
import tensorflow as tf

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, datasets
from tensorflow.keras.models import Sequential
from keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

## **Data Preprocessing**

In [7]:
data_directory = '../data/artists/artists/resized'
img_size = (150, 150)  

def load_data(data_dir):
    images = []
    labels = []

    for filename in os.listdir(data_dir):
        artist_name = re.sub(r'\d+', '', filename.strip('.jpg'))   

        img_path = os.path.join(data_dir, filename)
        img = Image.open(img_path).convert('RGB')
        img = img.resize(img_size)
        img_array = np.array(img)

        images.append(img_array)
        labels.append(artist_name)
    
    images_array = np.array(images)
    images_normalized = images_array / 255

    return images_normalized, np.array(labels)

images, labels = load_data(data_directory)

In [8]:
"""
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels)

one_hot_labels = to_categorical(numeric_labels)
"""
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [9]:
# Konvertierung der Numpy-Arrays in PyTorch-Tensoren
train_images_tensor = torch.from_numpy(train_images).float()
train_labels_tensor = torch.from_numpy(train_labels).long()
test_images_tensor = torch.from_numpy(test_images).float()
test_labels_tensor = torch.from_numpy(test_labels).long()

# Erstellen des Trainings- und Test-Datasets
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

# Definiere die Batch-Größe für das Training
batch_size = 32

# Erstellen der DataLoader für Trainings- und Testsets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## **CNN Model**

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [10]:
model = models.resnet50(pretrained=True)

# Einfrieren der Gewichte der Basis (ResNet50) Layers, um sie während des Trainings nicht zu aktualisieren
for param in model.parameters():
    param.requires_grad = False

# Ersetzen des Klassifikations-Head
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(set(labels)))  # Anpassen der Anzahl der Klassen basierend auf den Labels

# Verwende die GPU, falls verfügbar
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Definition des Optimierers und Verlustfunktion
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Training des Modells
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}")

# Evaluation des trainierten Modells
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

c:\Users\UserNA6153\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\UserNA6153\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\UserNA6153/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:12<00:00, 7.89MB/s]


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[32, 150, 150, 3] to have 3 channels, but got 150 channels instead